<a href="https://colab.research.google.com/github/aknakshay/VideoAnalytics/blob/master/Emotion_Classifier_Modelling_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emotion Classifier


In [0]:
#import libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

## Load the fer2013 grayscale face emotion dataset

https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data

We manually do an 80/20 train-test split and cache the data to disk.

In [0]:
def load_fer2013(force=False):
    #This function is taken from weights and biases demo
    """Load the emotion dataset"""
    if force or not os.path.exists("fer2013"):
        print("Downloading the face emotion dataset...")
        subprocess.check_output(
            "curl -SL https://www.dropbox.com/s/opuvvdv3uligypx/fer2013.tar | tar xz", shell=True)
    print("Loading dataset...")
    if not os.path.exists('face_cache.npz'):
        data = pd.read_csv("fer2013/fer2013.csv")
        pixels = data['pixels'].tolist()
        width, height = 48, 48
        faces = []
        for pixel_sequence in pixels:
            pixs = pixel_sequence.split(' ')
            try:
                face = np.asarray(pixel_sequence.split(
                    ' '), dtype=np.uint8).reshape(width, height)
                face = cv2.resize(face.astype('uint8'), (width, height))
                faces.append(face.astype('float32'))
            except ValueError:
              print("Unable to load face.")

        faces = np.asarray(faces)
        faces = np.expand_dims(faces, -1)
        emotions = pd.get_dummies(data['emotion']).as_matrix()

        val_faces = faces[int(len(faces) * 0.8):]
        val_emotions = emotions[int(len(faces) * 0.8):]
        train_faces = faces[:int(len(faces) * 0.8)]
        train_emotions = emotions[:int(len(faces) * 0.8)]
        np.savez('face_cache.npz', train_faces=train_faces, train_emotions=train_emotions,
                 val_faces=val_faces, val_emotions=val_emotions)
    cached = np.load('face_cache.npz')

    return cached['train_faces'], cached['train_emotions'], cached['val_faces'], cached['val_emotions']

# Model Training

In [0]:
import tensorflow as tf
tf.keras.backend.clear_session()


# Load dataset
input_shape = (48, 48, 1)
train_faces, train_emotions, val_faces, val_emotions = load_fer2013()
num_samples, num_classes = train_emotions.shape


In [0]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

model = tf.keras.Sequential()

model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3, 3), padding = 'same', 
                  activation ='relu', input_shape = input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Dropout(0.3))


model.add(tf.keras.layers.Conv2D(filters = 128, kernel_size = (3, 3), padding = 'same', 
                  activation ='relu', input_shape = input_shape))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Dropout(0.3))



model.add(tf.keras.layers.Conv2D(filters = 256, kernel_size = (3, 3), padding = 'same', 
                  activation ='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3)))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(256, activation="relu"))
model.add(tf.keras.layers.Dropout(0.3))

model.add(tf.keras.layers.Dense(64, activation="relu"))

model.add(tf.keras.layers.Dropout(0.4))
model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])


In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=.1)

seed = 1
training_generator = datagen.flow(train_faces, train_emotions, batch_size=32,subset='training',seed=seed)
validation_generator = datagen.flow(train_faces, train_emotions, batch_size=32,subset='validation',seed=seed)


In [0]:
model.fit_generator(datagen.flow(train_faces, train_emotions, batch_size=32),
                    steps_per_epoch=len(train_faces)/4, epochs=50,validation_data=validation_generator, validation_steps=(len(train_faces)*0.2)//32)

Epoch 1/50
7178/7177 [==============================] - 123s 17ms/step - loss: 2.0514 - acc: 0.2246 - val_loss: 1.8376 - val_acc: 0.2485
Epoch 2/50
7178/7177 [==============================] - 126s 18ms/step - loss: 1.8197 - acc: 0.2509 - val_loss: 1.8134 - val_acc: 0.2476
Epoch 3/50
7178/7177 [==============================] - 125s 17ms/step - loss: 1.7963 - acc: 0.2562 - val_loss: 1.7449 - val_acc: 0.2782
Epoch 4/50
7178/7177 [==============================] - 126s 18ms/step - loss: 1.7318 - acc: 0.2871 - val_loss: 1.6700 - val_acc: 0.3038
Epoch 5/50
7178/7177 [==============================] - 125s 17ms/step - loss: 1.6725 - acc: 0.3162 - val_loss: 1.6045 - val_acc: 0.3477
Epoch 6/50
7178/7177 [==============================] - 124s 17ms/step - loss: 1.6207 - acc: 0.3527 - val_loss: 1.5460 - val_acc: 0.3933
Epoch 7/50
7178/7177 [==============================] - 117s 16ms/step - loss: 1.5779 - acc: 0.3815 - val_loss: 1.4866 - val_acc: 0.4281
Epoch 8/50
7178/7177 [===================

In [0]:
model.save('emotion.h5')